In [1]:
%load_ext autoreload
%autoreload 2
%pwd

'/home/luca/uni/master/dl-in-vc/voxl3r'

In [2]:
# setup PYTHONPATH
import sys
sys.path += ['.', './extern/scannetpp', './extern/mast3r', './extern/mast3r/dust3r']

In [3]:
from pathlib import Path
import torch
from torchvision.io import read_image

import pyvista as pv
pv.set_jupyter_backend('client')

In [4]:
pv.start_xvfb()

In [17]:
import os
from models.surface_net_3d.run import visualize_run

run_name = '62hctrps'

ckpt_folder = list(Path("./.lightning/surface-net-3d/surface-net-3d/").glob("*"))
ckpt_folder = sorted(ckpt_folder, key=os.path.getmtime)
last_ckpt_folder = ckpt_folder[-1]
run_name = last_ckpt_folder.stem

visualize_run(run_name, 2, 0, show = ["train"])

No best_ckpts.pt found, falling back to last checkpoint
No data_config found in checkpoint, using default
First image of training is: /home/luca/mnt/data/scannetpp/data/0cf2e9402d/iphone/rgb/frame_006420.jpg
Chunk size was [1.28 1.28 1.28], resolution was 0.02, center was [0.         0.         1.47199988]
Grid size was 64
First image of validation is: /home/luca/mnt/data/scannetpp/data/0cf2e9402d/iphone/rgb/frame_004590.jpg
Chunk size was [1.28 1.28 1.28], resolution was 0.02, center was [0.         0.         1.47199988]
First image of test is: /home/luca/mnt/data/scannetpp/data/0cf2e9402d/iphone/rgb/frame_006820.jpg
Chunk size was [1.28 1.28 1.28], resolution was 0.02, center was [0.         0.         1.47199988]


Widget(value='<iframe src="http://localhost:37499/index.html?ui=P_0x7a41a13011f0_10&reconnect=auto" class="pyv…

AttributeError: 
This plotter has not yet been set up and rendered with ``show()``.
Consider setting ``off_screen=True`` for off screen rendering.


{'frame_000000.jpg': {'R_cw': array([[-0.96037249,  0.25331611, -0.11625677],
         [ 0.23399095,  0.50613997, -0.83010274],
         [-0.1514362 , -0.82441087, -0.54535658]]),
  't_cw': array([[ 3.83153],
         [-0.42915],
         [ 2.59675]]),
  'T_cw': array([[-0.96037249,  0.25331611, -0.11625677,  3.83153   ],
         [ 0.23399095,  0.50613997, -0.83010274, -0.42915   ],
         [-0.1514362 , -0.82441087, -0.54535658,  2.59675   ],
         [ 0.        ,  0.        ,  0.        ,  1.        ]]),
  'K': array([[1.43945e+03, 0.00000e+00, 9.58667e+02],
         [0.00000e+00, 1.44116e+03, 7.20891e+02],
         [0.00000e+00, 0.00000e+00, 1.00000e+00]]),
  'dist_coeffs': array([ 6.15734e-02, -7.52698e-02, -5.13368e-04, -7.38036e-05]),
  'width': 1920.0,
  'height': 1440.0},
 'frame_000010.jpg': {'R_cw': array([[-0.96587469,  0.23340676, -0.11228257],
         [ 0.21976322,  0.50908294, -0.83218909],
         [-0.13707741, -0.82846596, -0.54300455]]),
  't_cw': array([[ 3.87921

In [ ]:
import numpy as np
from dataset import SceneDataset
from models.surface_net_3d.data import SurfaceNet3DDataConfig
from utils.chunking import create_chunk, mesh_2_voxels
from utils.data_parsing import load_yaml_munch
from utils.transformations import invert_pose
from utils.visualize import visualize_mesh

config = load_yaml_munch("./utils/config.yaml")
data_config = SurfaceNet3DDataConfig(data_dir=config.data_dir, batch_size=16, num_workers=11, force_prepare=True)

resolution = data_config.grid_resolution
grid_size = data_config.grid_size
max_seq_len = data_config.seq_len

resolution = 0.02
chunk_size = resolution * grid_size.astype(np.float32)
center = np.array([0.0, 0.0, chunk_size[2]])

dataset = SceneDataset(
            camera=data_config.camera,
            data_dir=data_config.data_dir,
            scenes=data_config.scenes,
        )

data = dataset[0]
mesh = data["mesh"]
path_images = data["path_images"]
camera_params = data["camera_params"]

image_name = "DSC06230.JPG"

data_chunk = create_chunk(mesh.copy(), image_name, camera_params, max_seq_len=1, image_path=path_images, center=center, size=chunk_size, with_backtransform=True)

T_cw = camera_params[image_name]['T_cw']
_, _, T_wc = invert_pose(T_cw[:3,:3], T_cw[:3,3])

mesh_chunk_backtransformer = data_chunk["backtransformed"]
mesh_chunk = data_chunk["mesh"]
image_names_chunk = data_chunk["image_names"]
camera_params_chunk = data_chunk["camera_params"]
p_center = data_chunk["p_center"]

print(f"Chunk size: {chunk_size}")
print(f"Center: {center}")
print(f"Resolution: {resolution}")

Chunk size: [1.28 1.28 1.28]
Center: [0.         0.         1.27999997]
Resolution: 0.02


In [32]:
from models.surface_net_3d.data import VoxelGridDataset
from utils.chunking import mesh_2_local_voxels

image_names = list(camera_params.keys())

image_path = path_images
seq_len = data_config.seq_len

image_name = image_names[151 * data_config.seq_len]
data_chunk = create_chunk(
mesh.copy(),
image_name,
camera_params,
max_seq_len=seq_len,
image_path=image_path,
size=chunk_size,
center=center,
)

transformation = data_chunk["camera_params"][image_name]["T_cw"]
_, _, T_wc = invert_pose(transformation[:3, :3], transformation[:3, 3])

voxel_grid, coordinate_grid, occupancy_grid = mesh_2_local_voxels(
data_chunk["mesh"],
center=center,
pitch=resolution,
final_dim=grid_size[0],
to_world_coordinates=T_wc,
)



ValueError: Bounds must be (n, dimension * 2)!

In [30]:
voxel_grid, coordinates, occupancy_values = mesh_2_voxels(mesh_chunk, voxel_size=0.02, to_world_coordinates=T_wc)

visualize_mesh(voxel_grid.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)

Widget(value='<iframe src="http://localhost:35153/index.html?ui=P_0x735d91bee5a0_8&reconnect=auto" class="pyvi…

In [14]:
from utils.chunking import mesh_2_local_voxels
from utils.transformations import invert_pose

voxel_grid_local, coordinates_local, occupancy_values_local  = mesh_2_local_voxels(mesh_chunk_backtransformer,  center, 0.02, grid_size[0])
visualize_mesh(voxel_grid_local.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)

AttributeError: 'NoneType' object has no attribute 'encoding'

In [ ]:
from utils.chunking import mesh_2_local_voxels
from utils.transformations import invert_pose


T_cw = camera_params[image_name]['T_cw']
_, _, T_wc = invert_pose(T_cw[:3,:3], T_cw[:3,3])

voxel_grid_local, coordinates_local, occupancy_values_local  = mesh_2_local_voxels(mesh_chunk,  center, 0.02, grid_size[0])
visualize_mesh(voxel_grid_local.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)